In [ ]:
# 구글 드라이브 연결

from google.colab import drive
import os

drive.mount('/content/drive')

project_path = '/content/drive/MyDrive/ch6'
os.makedirs(project_path, exist_ok=True)

# 현재 경로를 /content/drive로 설정
os.chdir(project_path)
print("현재 작업 디렉토리:", os.getcwd())

Mounted at /content/drive
현재 작업 디렉토리: /content/drive/MyDrive/ch6


In [ ]:
!apt-get update && apt-get install -y gcc
!pip install angr

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,326 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,160 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,449 kB]
Hit:13 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:14 http://arc

In [13]:
!gcc -fno-stack-protector -z execstack -no-pie -o bof bof.c
!./bof

Enter the input (type 'quit' to exit): sdf
Conditions not met. Buffer overflow did not occur.
Enter the input (type 'quit' to exit): AZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd
All conditions met! Executing vulnerable function...
Buffer contents: AZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZZsecret123ddd


In [39]:
with open("bof_symbolic.py", "r") as f:
  file_contents = f.read()
  print(file_contents)

import angr
import claripy

# 바이너리 파일 경로 설정 (bof 프로그램이 컴파일된 위치)
binary_path = "./bof"

# 프로젝트 생성
project = angr.Project(binary_path, auto_load_libs=False)

# 입력값의 길이 설정 (입력값 길이는 최소 17 이상이어야 조건을 만족)
input_length = 30  # 입력값의 길이 설정 (프로그램의 조건에 따라 조정)

# Angr의 BitVector를 사용하여 심볼릭 변수로 입력값을 설정
input_chars = [claripy.BVS(f'input_{i}', 8) for i in range(input_length)]
input_expr = claripy.Concat(*input_chars)  # 입력값을 하나의 심볼릭 표현식으로 결합

# 초기 상태 설정 (args에 심볼릭 입력값 전달)
state = project.factory.full_init_state(
    args=[binary_path],  # 프로그램 실행 인자에 바이너리 경로 설정
    stdin=angr.SimFileStream(name="stdin", content=input_expr, has_end=False),  # SimFileStream 사용
    add_options={angr.options.LAZY_SOLVES, angr.options.ZERO_FILL_UNCONSTRAINED_MEMORY, angr.options.ZERO_FILL_UNCONSTRAINED_REGISTERS}
)

# 심볼릭 메모리의 stdin에 입력값을 설정하고 ASCII 값 범위로 제한
for byte in input_chars:
    state.solver.add(byte >= 0x20, byte <= 0x7e)  # ASCII printable 문자로 제한

# 시뮬레이션 매니저 생성
simgr = project.factory.simulation_manager(state)



In [40]:
!python bof_symbolic.py

[State ID: Unknown]
Program counter (PC): 4198704
[State ID: Unknown]
Program counter (PC): 5242880
[State ID: Unknown]
Program counter (PC): 4198400
[State ID: Unknown]
Program counter (PC): 4198422
[State ID: Unknown]
Program counter (PC): 6295632
[State ID: Unknown]
Program counter (PC): 4198928
[State ID: Unknown]
Program counter (PC): 4198816
[State ID: Unknown]
Program counter (PC): 4198864
[State ID: Unknown]
Program counter (PC): 6295632
[State ID: Unknown]
Program counter (PC): 4199449
[State ID: Unknown]
Program counter (PC): 4198624
[State ID: Unknown]
Program counter (PC): 5242920
[State ID: Unknown]
Program counter (PC): 4199481
[State ID: Unknown]
Program counter (PC): 4198656
[State ID: Unknown]
Program counter (PC): 5242936
[State ID: Unknown]
Program counter (PC): 4199505
[State ID: Unknown]
Program counter (PC): 4198640
[State ID: Unknown]
Program counter (PC): 5242928
[State ID: Unknown]
Program counter (PC): 4199527
[State ID: Unknown]
Program counter (PC): 4198672


In [41]:
!echo "Asecret123@ @ Z@@@    @ @ @n@~" | ./bof

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z@@@    @ @ @n@~
Enter the input (type 'quit' to exit): All conditions met! Executing vulnerable function...
Buffer contents: Asecret123@ @ Z